In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('restaurants_visitors.csv')
df

,id,visit_date,visit_datetime,reserve_datetime,reserve_visitors
0,db80363d35f10926,1/1/2016,1/1/2016 0:00,1/1/2016 1:00,5
1,db80363d35f10926,1/2/2016,1/2/2016 1:00,1/1/2016 16:00,2
2,db80363d35f10926,1/2/2016,1/2/2016 1:00,1/1/2016 15:00,4
3,db80363d35f10926,1/2/2016,1/2/2016 18:00,1/2/2016 14:00,2
4,db80363d35f10926,1/2/2016,1/2/2016 18:00,1/2/2016 2:00,3
...,...,...,...,...,...
12157,3cad29d1a23209d2,5/27/2017,5/27/2017 21:00,2/9/2017 18:00,3
12158,1033310359ceeac1,5/28/2017,5/28/2017 23:00,4/15/2017 22:00,2
12159,1033310359ceeac1,5/28/2017,5/28/2017 23:00,4/18/2017 17:00,2
12160,1033310359ceeac1,5/30/2017,5/30/2017 23:00,4/19/2017 17:00,8


In [3]:
df.describe()

,reserve_visitors
count,12162.000000
mean,4.040701
std,4.141749
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,56.000000


In [4]:
df.dtypes

id                  object
visit_date          object
visit_datetime      object
reserve_datetime    object
reserve_visitors     int64
dtype: object

In [5]:
df['visit_datetime'] = pd.to_datetime(df['visit_datetime'])
df['visit_date'] = pd.to_datetime(df['visit_date'])
df.dtypes

id                          object
visit_date          datetime64[ns]
visit_datetime      datetime64[ns]
reserve_datetime            object
reserve_visitors             int64
dtype: object

In [6]:
monthly_visits = df.groupby(df['visit_datetime'].dt.to_period('M'))['reserve_visitors'].sum().reset_index()
monthly_visits

,visit_datetime,reserve_visitors
0,2016-01,906
1,2016-02,868
2,2016-03,1307
3,2016-04,1340
4,2016-05,833
5,2016-06,1205
6,2016-07,1280
7,2016-09,2
8,2016-10,901
9,2016-11,4809


In [7]:
monthly_counts = df.groupby(df['visit_datetime'].dt.to_period('M')).size()
monthly_counts

visit_datetime
2016-01     203
2016-02     202
2016-03     269
2016-04     277
2016-05     169
2016-06     182
2016-07     264
2016-09       1
2016-10     239
2016-11    1323
2016-12    2324
2017-01    1551
2017-02    1707
2017-03    2024
2017-04    1335
2017-05      92
Freq: M, dtype: int64

In [8]:
monthly_restaurants = df.groupby(df['visit_datetime'].dt.to_period('M'))['id'].nunique()
monthly_restaurants

visit_datetime
2016-01     5
2016-02     5
2016-03     6
2016-04     7
2016-05     6
2016-06     3
2016-07     7
2016-09     1
2016-10    28
2016-11    30
2016-12    31
2017-01    29
2017-02    30
2017-03    30
2017-04    29
2017-05    19
Freq: M, Name: id, dtype: int64

# Primer Modelado 'Regresion Lineal'

In [9]:
# Creación Feature 'month'
monthly_visits['month'] = monthly_visits['visit_datetime'].dt.month

In [10]:
# Crear modelo
model = LinearRegression()

In [11]:
# Entrenar modelo
model.fit(monthly_visits[['month']], monthly_visits['reserve_visitors'])

LinearRegression()

In [12]:
# Generar fechas para los siguientes 6 meses
last_real_month = monthly_visits['visit_datetime'].max().to_timestamp()
future_months = pd.date_range(start=last_real_month + pd.DateOffset(months=1), periods=6, freq='M')

In [13]:
# Creacion de Nuevo df para las futuras predicciones
future_predictions_df = pd.DataFrame({'visit_datetime': future_months, 'month': future_months.month})

In [14]:
# Predecir
future_predictions = model.predict(future_predictions_df[['month']])

In [15]:
# Creación de df de Resultados
results = pd.DataFrame({'visit_datetime': future_predictions_df['visit_datetime'],'Predicted Visitors': future_predictions})
results

,visit_datetime,Predicted Visitors
0,2017-06-30,3130.711976
1,2017-07-31,3216.929396
2,2017-08-31,3303.146816
3,2017-09-30,3389.364236
4,2017-10-31,3475.581656
5,2017-11-30,3561.799076


En comparación con los mismos del año anterior, su % de crecmiento mensual y las anomalias que se presentan en los meses de agosto y septiembre del 2016 y mayo del 2017 se descarta el uso de una regresion lineal simple para esta predicción

# Regresion Lineal (Feature Engineering)
Como primer paso se hará un cambio en los totales del mes de agosto (08) y septiembre(09) del 2016 y de mayo (05) del 2017 rellenando con un promedio de los 7 meses anteriores con un ajuste tomando en cuenta el promedio del porcentaje de crecimiento entre mes.

In [16]:
monthly_visits

,visit_datetime,reserve_visitors,month
0,2016-01,906,1
1,2016-02,868,2
2,2016-03,1307,3
3,2016-04,1340,4
4,2016-05,833,5
5,2016-06,1205,6
6,2016-07,1280,7
7,2016-09,2,9
8,2016-10,901,10
9,2016-11,4809,11


In [17]:
# Crear la nueva fila que deseas insertar
ago_16 = pd.DataFrame({'visit_datetime': ['2016-08'], 'reserve_visitors': [0], 'month': [8]})

# Insertar la nueva fila entre los registros 6 y 7
monthly_visits = pd.concat([monthly_visits.iloc[:7], ago_16, monthly_visits.iloc[7:]], ignore_index=True)
monthly_visits

,visit_datetime,reserve_visitors,month
0,2016-01,906,1
1,2016-02,868,2
2,2016-03,1307,3
3,2016-04,1340,4
4,2016-05,833,5
5,2016-06,1205,6
6,2016-07,1280,7
7,2016-08,0,8
8,2016-09,2,9
9,2016-10,901,10


Se hara un cambio en los registros con anomalias siguiendo ciertos procesos
1. Promedio ultimos 7 meses
2. Crecimiento mensual 6 meses
3. Verificación de estacinoalidad con el año pasado
4. Promedio de registros ultimos 7 meses
5. Crecimiento de registros primeros meses del 2016
6. Crecimiento de registros primeros meses del 2017
7. Relación Registros con Total de Visitantes
8. Verificación de estacinoalidad con el año pasado


In [18]:
monthly_visits_2 = monthly_visits.copy()
monthly_visits_2

,visit_datetime,reserve_visitors,month
0,2016-01,906,1
1,2016-02,868,2
2,2016-03,1307,3
3,2016-04,1340,4
4,2016-05,833,5
5,2016-06,1205,6
6,2016-07,1280,7
7,2016-08,0,8
8,2016-09,2,9
9,2016-10,901,10


In [19]:
monthly_visits_2.loc[7, 'reserve_visitors'] = 1220
monthly_visits_2.loc[8, 'reserve_visitors'] = 1279
monthly_visits_2.loc[16, 'reserve_visitors'] = 5225
monthly_visits_2

,visit_datetime,reserve_visitors,month
0,2016-01,906,1
1,2016-02,868,2
2,2016-03,1307,3
3,2016-04,1340,4
4,2016-05,833,5
5,2016-06,1205,6
6,2016-07,1280,7
7,2016-08,1220,8
8,2016-09,1279,9
9,2016-10,901,10


In [20]:
results

,visit_datetime,Predicted Visitors
0,2017-06-30,3130.711976
1,2017-07-31,3216.929396
2,2017-08-31,3303.146816
3,2017-09-30,3389.364236
4,2017-10-31,3475.581656
5,2017-11-30,3561.799076


In [21]:
# Entrenar modelo
model.fit(monthly_visits_2[['month']], monthly_visits_2['reserve_visitors'])

# Creacion de Nuevo df para las futuras predicciones
future_predictions_df = pd.DataFrame({'visit_datetime': future_months, 'month': future_months.month})

# Predecir
future_predictions = model.predict(future_predictions_df[['month']])

# Creación de df de Resultados
results2 = pd.DataFrame({'visit_datetime': future_predictions_df['visit_datetime'],'Predicted Visitors': future_predictions})
results2

,visit_datetime,Predicted Visitors
0,2017-06-30,3355.940647
1,2017-07-31,3426.828537
2,2017-08-31,3497.716427
3,2017-09-30,3568.604317
4,2017-10-31,3639.492206
5,2017-11-30,3710.380096


# ARIMA con Datos Originales

In [22]:
future_predictions_df

,visit_datetime,month
0,2017-06-30,6
1,2017-07-31,7
2,2017-08-31,8
3,2017-09-30,9
4,2017-10-31,10
5,2017-11-30,11


In [23]:
(monthly_visits['reserve_visitors'])

0      906
1      868
2     1307
3     1340
4      833
5     1205
6     1280
7        0
8        2
9      901
10    4809
11    9785
12    5998
13    6077
14    8152
15    5228
16     452
Name: reserve_visitors, dtype: int64

In [24]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Crear el modelo ARIMA
model = ARIMA(monthly_visits['reserve_visitors'], order=(5, 1, 0))

# Entrenar el modelo
model_fit = model.fit()

# Realizar las predicciones para los siguientes 6 meses
forecast = model_fit.forecast(steps=6)

# Crear un DataFrame para los resultados
results3  = pd.DataFrame({'visit_datetime': monthly_visits['visit_datetime'], 'Predicted Visitors': forecast})
results3

,visit_datetime,Predicted Visitors
0,2016-01,NaN
1,2016-02,NaN
2,2016-03,NaN
3,2016-04,NaN
4,2016-05,NaN
5,2016-06,NaN
6,2016-07,NaN
7,2016-08,NaN
8,2016-09,NaN
9,2016-10,NaN


In [25]:
monthly_visits_2['reserve_visitors']

0      906
1      868
2     1307
3     1340
4      833
5     1205
6     1280
7     1220
8     1279
9      901
10    4809
11    9785
12    5998
13    6077
14    8152
15    5228
16    5225
Name: reserve_visitors, dtype: int64

# ARIMA con Feature Engineering

In [26]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Crear el modelo ARIMA
model = ARIMA(monthly_visits_2['reserve_visitors'], order=(5, 1, 0))

# Entrenar el modelo
model_fit = model.fit()

# Realizar las predicciones para los siguientes 6 meses
forecast = model_fit.forecast(steps=6)

# Crear un DataFrame para los resultados
results4  = pd.DataFrame({'visit_datetime': monthly_visits_2['visit_datetime'], 'Predicted Visitors': forecast})
results4

,visit_datetime,Predicted Visitors
0,2016-01,NaN
1,2016-02,NaN
2,2016-03,NaN
3,2016-04,NaN
4,2016-05,NaN
5,2016-06,NaN
6,2016-07,NaN
7,2016-08,NaN
8,2016-09,NaN
9,2016-10,NaN


In [62]:
monthly_visits_2

,visit_datetime,reserve_visitors,month
0,2016-01,906,1
1,2016-02,868,2
2,2016-03,1307,3
3,2016-04,1340,4
4,2016-05,833,5
5,2016-06,1205,6
6,2016-07,1280,7
7,2016-08,1220,8
8,2016-09,1279,9
9,2016-10,901,10


# ARIMA con TimeSeriesSplit (TSS) cross validation

In [74]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear el modelo ARIMA
orders_to_try = [(5, 1, 0), (1, 1, 1)]  

# Dividir los datos en conjunto de entrenamiento y prueba
train_size = 8
train, test = monthly_visits_2['reserve_visitors'][:train_size], monthly_visits_2['reserve_visitors'][train_size:]

# Iniciar el objeto TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=4)

best_rmse = float('inf')
best_order = None

# Bucle para probar diferentes combinaciones de hiperparámetros
for order in orders_to_try:
    total_rmse = 0
    
    # Bucle de validación cruzada
    for train_index, test_index in tscv.split(train):
        train_fold, test_fold = train.iloc[train_index], train.iloc[test_index]
        history = train_fold.tolist()
        fold_predictions = []

        # Realizar la predicción para el conjunto de prueba actual
        for t in range(len(test_fold)):
            model = ARIMA(history, order=order)
            model_fit = model.fit()
            forecast = model_fit.forecast(steps=1)
            fold_predictions.append(forecast[0])
            history.append(test_fold.iloc[t])

        # Calcular RMSE para esta iteración
        rmse = np.sqrt(mean_squared_error(test_fold, fold_predictions))
        total_rmse += rmse

    # Calcular el promedio RMSE para todas las divisiones de validación cruzada
    avg_rmse = total_rmse / tscv.get_n_splits()

    # Actualizar el mejor conjunto de hiperparámetros si es necesario
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_order = order

# Imprimir el conjunto de hiperparámetros que produce el menor RMSE
print("Mejor conjunto de hiperparámetros:", best_order)
print("Menor RMSE promedio:", best_rmse)


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax

Mejor conjunto de hiperparámetros: (1, 1, 1)
Menor RMSE promedio: 315.80114172922487


# ARIMA con Hiperparametros (1,1,1) y Datos Originales

In [59]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Crear el modelo ARIMA
model = ARIMA(monthly_visits['reserve_visitors'], order=(1, 1, 1))

# Entrenar el modelo
model_fit = model.fit()

# Realizar las predicciones para los siguientes 6 meses
forecast = model_fit.forecast(steps=6)

# Crear un DataFrame para los resultados
results5  = pd.DataFrame({'visit_datetime': monthly_visits['visit_datetime'], 'Predicted Visitors': forecast})
results5

,visit_datetime,Predicted Visitors
0,2016-01,NaN
1,2016-02,NaN
2,2016-03,NaN
3,2016-04,NaN
4,2016-05,NaN
5,2016-06,NaN
6,2016-07,NaN
7,2016-08,NaN
8,2016-09,NaN
9,2016-10,NaN


# ARIMA con Hiperparametros (1,1,1) y Feature Engineering

In [60]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Crear el modelo ARIMA
model = ARIMA(monthly_visits_2['reserve_visitors'], order=(1, 1, 1))

# Entrenar el modelo
model_fit = model.fit()

# Realizar las predicciones para los siguientes 6 meses
forecast = model_fit.forecast(steps=6)

# Crear un DataFrame para los resultados
results6  = pd.DataFrame({'visit_datetime': monthly_visits_2['visit_datetime'], 'Predicted Visitors': forecast})
results6

,visit_datetime,Predicted Visitors
0,2016-01,NaN
1,2016-02,NaN
2,2016-03,NaN
3,2016-04,NaN
4,2016-05,NaN
5,2016-06,NaN
6,2016-07,NaN
7,2016-08,NaN
8,2016-09,NaN
9,2016-10,NaN
